In [ ]:
# 1. PREORDER
class Solution:
# 1.1. for:
  def fib_for(self, n:int) -> int:
    if n == 0 or n == 1:
      return n
    dp = [0,1]
    for i in range(2,n):
      dp.append(dp[i-1],dp[i-2])
    return dp[n]


In [ ]:
class Solution_BFS:
  # 1.2. BFS:
  def fib_bfs(self, n:int) -> int:
    # *. edge cases
    if n == 0 or n == 1:
      return n
    
    # 0. construct the array to store results/values
    dp = [0]*(n+1)
    dp[1] = 1

    # 1. construct the QUEUE: only stores KNOWN node 
    queue = [0,1]

    # 2. Get in-degree array
    indegree = [2]*(n+1)
    indegree[0] = 0
    indegree[1] = 0

    while queue:
      # 3. popout KNOWN node out of queue
      x = queue.pop(0)

      # 4. expansion
      if x == n:
        break
      next = [2] if x == 0 else [x+1, x+2]
      for y in next:
        if y > n: continue # when x = n, 
        dp[y] += dp[x]
        indegree[y] -= 1
        # 5. put node with INDEGREE==0 INTO queue
        if indegree[y] == 0: 
          queue.append(y)

    return dp[n]


  def fib_bfs_spaceSaving(self, n:int) -> int:
    # *. edge cases
    if n == 0 or n == 1: return n
  
    # 0. construct the array to store results/values
    dp = [0,1,0]

    # 1. construct the QUEUE: only stores KNOWN node 
    queue = [0,1]

    # 2. Get in-degree array
    indegree = [0,0,2]

    while queue:
      # 3. popout KNOWN node out of queue
      x = queue.pop(0)
      # 4. expansion
      if x == n: break
      next = [2] if x == 0 else [x+1, x+2]
      for y in next:
        if y > n: continue # when x = n-1 
        dp[y%3] += dp[x%3]
        indegree[y%3] -= 1
        # 5. put node with INDEGREE==0 INTO queue
        if indegree[y%3] == 0: 
          queue.append(y)
      
      # when all operations done for current node (left waiting queue)
      # dont need it anymore
      dp[x%3] = 0
      indegree[x%3] = 2

    return dp[n%3]

In [ ]:
class Solution_DFS_preorder:
    
    def fib(self, n:int) -> int:
        if n < 2: return n

        # 0. 构建数据存储结构
        dp = [0]*(n+1)
        dp[1] = 1
        # 1. 统计入度
        indeg = [2]*(n+1)
        indeg[0], indeg[1] = 0, 0
        # 2. 构建初始队列
        queue = [0,1]
        # q = queue.Queue()
        # for i in [0,1]:
        #   q.put(i)

        # 3. 出队  -- 明确目标状态 ！！！ x == n 
        def dfs(x):
          if x == n: return
          # 4. 扩展
          next = [2] if x==0 else [x+1,x+2]
          for y in next:
             if y > n: continue
             dp[y] += dp[x]
             indeg[y] -= 1
             # 5. 入队
             if indeg[y] == 0: 
                dfs(y)
        
        # DFS2. 手动模拟栈，实现dfs先序：
        def dfs_stack(x):
           if x == n: return

           stack =[[x, 0]] #[number, visit_count]
           while stack:
            cur, vis = stack[-1]
            case1 = cur == 0 and vis == 1 # 0，并且访问过了（给2），完成使命
            case2 = cur != 0 and vis == 2 # 非0，且传值过两次（vis==2），完成使命

            # a. 扩展入栈 -》 先序传值
            if not case1 and not case2:
              next = cur+vis+1 if cur!=0 else 2 # 扩展节点+1： x+vis+1
              stack[-1][1] += 1 # visit_count of current node + 1
              if next > n: continue
              dp[next] += dp[cur]
              indeg[next] -= 1
              # 入栈
              if indeg[next] == 0:
                 stack.append([next,0])
            # b. 结束出栈 -》 后续传值 -》 出栈后将该值传递给栈顶节点
            else:
               stack.pop()

        for x in queue:
           dfs(x)

        return dp[n] 

In [ ]:
class Solution_DFS_postorder:
    # 2. Post order  --  DFS:

    def fib_dfs(self, n:int) -> int:
        if n<2: return n
        dp = [0]*(n+1)
        dp[0],dp[1] = 0,1

        # 2.1. 
        def dfs(i,dp):
          if i<=1: return i
          elif dp[i] != 0:
            return dp[i]
          else:
             dp[i] = dfs(i-1,dp) + dfs(i-2,dp)
             return dp[i]
        
        # 2.2. stack
        def dfs_stack(i, dp):
           stack = [[i,0]]
           returnVal = [0]*(n+1)
           while stack:
              cur,vis = stack[-1]
              invalid = cur == 0 or cur == 1 or ( cur > 1 and dp[cur] != 0)
              if invalid:
                 stack.pop()
                 returnVal[i] = dp[i]
              elif vis == 0:
                 stack.pop()
                 stack.append([cur,1])
                 stack.append([cur-1,0])
              elif vis == 1:
                stack.pop()
                stack.append([cur,2])
                stack.append([cur-2,0])
              elif vis == 2:
                stack.pop()
                dp[i] = returnVal[i-1] + returnVal[i-2]     
                returnVal[i] = dp[i]
           
        return dfs(n,dp)  # postorder, 从N开始
    
  

### 在斐波那契这道题目中用DFS实现这四种顺序的值传递
提示：每个顺序都对应一个值的传递，每个值的传递都对应着递推三要素，如果你不熟悉的话，请写出以下每个值传递的递推三要素，一定要写出来！！！如果理解上有困难，先分开写，再合起来

1. 先序*访问：cnt 用来统计节点个数，  
  1.1. 状态语义：count = 0边界，count = N目标  <br>
  1.2. 递推公式：count +1    <br>
  1.3. 顺序：入栈顺序？后->前？？？  <br>

2. 先序*邻接：next用来传递一个节点的后驱节点
  状态语义：next = null 边界，next = null目标  <br>
  递推公式：dp[i].next = dp[i+1]   <br>
  顺序：入栈顺序？后->前？？？  <br>

3. 后序*访问：topo 拓扑排序
  状态语义：topo is null边界，len(topo) = n 目标  <br>
  递推公式：topo.append(i)     <br>
  顺序：出栈顺序？前->后  <br>
  
4. **后序*邻接**：return val：斐波那契递推
  状态语义：dp[0],dp[1]边界，dp[n]目标  <br>
  递推公式：dp[i].val = dp[i-1].val + dp[i-2].val    <br>
  顺序：出栈顺序？  <br>


In [ ]:
class Node:
    def __init__(self,val):
        self.val = val
        self.prevs = []
        self.nexts = []

class Solution_DFS：
    def fib_value(self, n:int) -> int:
        if n<2: return n
        dp = [None]*(n+1)
        dp[0] = Node(0)
        dp[1] = Node(1)
        topo = []
        cnt = 0
        return self.dfs(n,None,dp,topo,cnt)

    def dfs(self,i,nxt,dp,topo,cnt):
        
        if dp[i] != None:
            dp[i].nexts.append(nxt)
            return dp[i].val
        dp[i] = Node(0)  #原值为None：首次遇到的index
        dp[i].nexts.append(nxt)
        cnt += 1
        self.dfs(i-1,dp[i],dp,topo,cnt)
        self.dfs(i-2,dp[i],dp,topo,cnt)
        dp[i].val = dp[i-1].val + dp[i-2].val
        topo.append(i)
        return dp[i].val